# Машинное обучение, РЭШ

## [Практическое задание 3. Градиентный спуск своими руками](https://www.youtube.com/watch?v=dQw4w9WgXcQ)

### Общая информация
Дата выдачи: 17.11.2022

Дедлайн: 23:59MSK 24.11.2022

### О задании

В данном задании необходимо реализовать обучение линейной регрессии с помощью различных вариантов градиентного спуска.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.


### Формат сдачи
Задания загружаются на my.nes. Присылать необходимо ноутбук с выполненным заданием. 

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

**Оценка**: ...

## Реализация градиентного спуска

Реализуйте линейную регрессию с функцией потерь MSE, обучаемую с помощью:

**Задание 1 (1 балл)** Градиентного спуска;

**Задание 2.1 (2 балла)** Стохастического градиентного спуска + Batch SGD;

**Задание 2.2 (2 балла)** SGD Momentum;

**Бонусное задание (2 балл)** Adagrad, RMSProp, Adam;

Во всех пунктах необходимо соблюдать следующие условия:

* Все вычисления должны быть векторизованы;
* Циклы средствами python допускается использовать только для итераций градиентного спуска;
* В качестве критерия останова необходимо использовать (одновременно):

    * проверку на евклидовую норму разности весов на двух соседних итерациях (например, меньше некоторого малого числа порядка $10^{-6}$, задаваемого параметром `tolerance`);
    * достижение максимального числа итераций (например, 10000, задаваемого параметром `max_iter`).
* Чтобы проследить, что оптимизационный процесс действительно сходится, будем использовать атрибут класса `loss_history` — в нём после вызова метода `fit` должны содержаться значения функции потерь для всех итераций, начиная с первой (до совершения первого шага по антиградиенту);
* Инициализировать веса можно случайным образом или нулевым вектором. 


Ниже приведён шаблон класса, который должен содержать код реализации каждого из методов.

In [ ]:
def pprint(*args, **kwargs):
    print(">>", *args, **kwargs)

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


class LinearReg(BaseEstimator):
    def __init__(self, gd_type='full', optimizer_type='full', loss_function="mse", eta_type="static",
                 tolerance=1e-4, max_iter=1e5, w0=None, eta=1e-2, batch_size=1, alpha_momentum=0.4, eps_adagrad=1e-3, alpha_rmsprop=0.99):
        """
        gd_type: 'full' or 'stochastic'
        tolerance: for stopping gradient descent
        max_iter: maximum number of steps in gradient descent
        w0: np.array of shape (d) - init weights
        eta: learning rate
        alpha: momentum coefficient
        """
        self.gd_type = gd_type
        self.eta_type = eta_type
        self.optimizer_type = optimizer_type
        self.loss_function = loss_function
        self.tolerance = tolerance
        self.max_iter = int(max_iter)
        self.w0 = w0
        self.w = None
        self.eta = eta
        self.batch_size = int(batch_size) # batch size for batch sgd
        self.loss_history = None # list of loss function values at each training iteration
        self.r2_history = None
        self.w_history = None # list of weights at each training iteration
        self.alpha_momentum = alpha_momentum
        self.eps_adagrad = eps_adagrad
        self.alpha_rmsprop = alpha_rmsprop
        self.final_iterations = 0

    def fit(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: self
        """
        self.loss_history = []
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

        # initialize vector of weights
        if self.w0 is None:
            self.w0 = np.zeros(X.shape[1])

        self.w = self.w0.copy()
        self.w_history = [self.w.copy()]
        self.loss_history = [self.calc_loss(X, y)]
        self.r2_history = [r2_score(y, self.predict(X))]

        eta_t = self.eta
        h_t_previous = np.zeros_like(self.w)

        # parameters for adagrad
        G_k = np.zeros_like(self.w)

        for iteration in tqdm(range(self.max_iter)):

            # manipulating step eta (for stochastic and stochastic batch gradient calculation)
            # note: robbins-monro conditions
            if self.eta_type == "dynamic":
                eta_t = self.eta / np.sqrt(iteration + 1)

            # manipulating the decrease of anti-gradient
            gradient = self.calc_gradient(X,y)
            if self.optimizer_type == "adagrad":
                G_k += gradient**2
                h_t = self.eta * gradient / np.sqrt(G_k + self.eps_adagrad)
            elif self.optimizer_type == "rmsprop":
                G_k = self.alpha_rmsprop * G_k + (1 - self.alpha_rmsprop) * gradient**2
                h_t = self.eta * gradient / np.sqrt(G_k + self.eps_adagrad)
            elif self.optimizer_type == "momentum":
                h_t = self.alpha_momentum * h_t_previous + eta_t * gradient
                h_t_previous = h_t
            elif self.optimizer_type == "full":
                h_t = eta_t * gradient
            else:
                raise Exception("Unknown optimizer type")

            self.w -= h_t
            self.w_history.append(self.w.copy())
            self.loss_history.append(self.calc_loss(X, y))
            self.r2_history.append(r2_score(y, self.predict(X)))
            # pprint(self.loss_history[-1])

            self.final_iterations = iteration

            if np.linalg.norm(self.w_history[-1] - self.w_history[-2]) <= self.tolerance:
                break

        # transform list to np.array
        self.w_history = np.array(self.w_history)

        return self

    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

        return X @ self.w

    def calc_gradient(self, X, y):
        """
        X: np.array of shape (ell, d) (ell can be equal to 1 if stochastic)
        y: np.array of shape (ell)
        ---
        output: np.array of shape (d)
        """
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

        if self.gd_type == "full":
            return 2 * X.T @ (X @ self.w - y) / y.shape[0]
        elif self.gd_type == "stochastic":
            sample = np.random.choice(X.shape[0], size=self.batch_size, replace=False)
            return 2 * X[sample].T @ (X[sample] @ self.w - y[sample]) / self.batch_size
        else:
            raise Exception("Unknown gd_type!")

    def calc_loss(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: float
        """
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        if self.loss_function == "mse":
            return mean_squared_error(y, self.predict(X))
        else:
            raise Exception("Unknown loss function!")

**Задание 3 (0 баллов)**
* Загрузите данные из домашнего задания 2 ([train.csv](https://www.kaggle.com/c/nyc-taxi-trip-duration/data));
* Разбейте выборку на обучающую и тестовую в отношении 7:3 с random_seed=0;
* Преобразуйте целевую переменную `trip_duration` как $\hat{y} = \log{(y + 1)}$.

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
df = pd.read_csv("data/nyc-taxi-trip-duration/train.csv")

In [ ]:
# mini eda
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])
df = df.drop(columns="dropoff_datetime")

df["log_trip_duration"] = np.log1p(df["trip_duration"])
df = df.drop(columns="trip_duration")

df["hour"] = df["pickup_datetime"].dt.hour
df["month"] = df["pickup_datetime"].dt.month
df["day_year_number"] = df["pickup_datetime"].dt.dayofyear

from haversine import haversine
def distance(x):
    return haversine((x["pickup_latitude"], x["pickup_longitude"]), (x["dropoff_latitude"], x["dropoff_longitude"]))

df["haversine"] = df.apply(distance, axis = 1)
df["log_haversine"] = np.log1p(df["haversine"])
df = df.drop(columns="haversine")

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=0)

# fix numerical features
numerical_f = ["month", "hour", "day_year_number", "log_haversine"]

x_train, y_train = train_df[numerical_f], train_df["log_trip_duration"].to_numpy()
x_test, y_test = test_df[numerical_f], test_df["log_trip_duration"].to_numpy()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from statsmodels.api import add_constant
from sklearn.pipeline import Pipeline

column_transformer = ColumnTransformer([("numeric", StandardScaler(), numerical_f)])
pipe = Pipeline([("column_transformer", column_transformer)])

x_train = add_constant(pipe.fit_transform(x_train))
x_test = add_constant(pipe.transform(x_test))

**Задание 4 (3 балла)**. Обучите и провалидируйте модели на данных из предыдущего пункта, сравните качество между методами по метрикам MSE и $R^2$. Исследуйте влияние параметров `max_iter` и `eta` на процесс оптимизации. Согласуется ли оно с вашими ожиданиями?

In [ ]:
from itertools import product
eta_list = [5e-2, 1e-1]
max_iter_list = [1e3, 2e3]

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
loss_list = []
for eta_i, max_iter_i in product(eta_list, max_iter_list):
    temp_class = LinearReg(optimizer_type="full", gd_type="full", max_iter=max_iter_i, eta_type="static", eta=eta_i) # full optimizer + full gradient
    temp_class.fit(X=x_train, y=y_train)
    loss_list.append(temp_class.loss_history)
    pprint(f"number of iterations: {temp_class.final_iterations}")
    pprint(f"loss function on test: {temp_class.calc_loss(X=x_test, y=y_test)}")
    pprint(f"r2 on test: {r2_score(y_test, temp_class.predict(x_test))}")

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
for i, (eta_i, max_iter_i) in enumerate(product(eta_list, max_iter_list)):
    fig.add_trace(go.Scatter(
        y=loss_list[i], mode="lines", name=f"eta={eta_i}, max_iter={int(max_iter_i)}"
    ))
fig.update_layout(title="Loss function for vanilla GD with different step and number of max iterations", xaxis_title="Iteration", yaxis_title="Loss (MSE)")
fig.show()

In [ ]:
# strangely, when eta is dynamic, the algorithm doesn't converge in 1000 iterations
loss_list = []
for eta_i, max_iter_i in product(eta_list, max_iter_list):
    temp_class = LinearReg(optimizer_type="full", gd_type="stochastic", max_iter=max_iter_i, eta=eta_i) # stochastic on 1 element
    temp_class.fit(X=x_train, y=y_train)
    loss_list.append(temp_class.loss_history)
    pprint(f"number of iterations: {temp_class.final_iterations}")
    pprint(f"loss function on test: {temp_class.calc_loss(X=x_test, y=y_test)}")
    pprint(f"r2 on test: {r2_score(y_test, temp_class.predict(x_test))}")

In [ ]:
fig = go.Figure()
for i, (eta_i, max_iter_i) in enumerate(product(eta_list, max_iter_list)):
    fig.add_trace(go.Scatter(
        y=loss_list[i], mode="lines", name=f"eta={eta_i}, max_iter={int(max_iter_i)}"
    ))
fig.update_layout(title="Loss function for SGD with different (static) step and number of max iterations", xaxis_title="Iteration", yaxis_title="Loss (MSE)")
fig.show()

In [ ]:
loss_list = []
for eta_i, max_iter_i in product(eta_list, max_iter_list):
    temp_class = LinearReg(optimizer_type="full", gd_type="stochastic", max_iter=max_iter_i, eta=eta_i, eta_type="dynamic") # stochastic on 1 element
    temp_class.fit(X=x_train, y=y_train)
    loss_list.append(temp_class.loss_history)
    pprint(f"number of iterations: {temp_class.final_iterations}")
    pprint(f"loss function on test: {temp_class.calc_loss(X=x_test, y=y_test)}")
    pprint(f"r2 on test: {r2_score(y_test, temp_class.predict(x_test))}")

In [ ]:
fig = go.Figure()
for i, (eta_i, max_iter_i) in enumerate(product(eta_list, max_iter_list)):
    fig.add_trace(go.Scatter(
        y=loss_list[i], mode="lines", name=f"eta={eta_i}, max_iter={int(max_iter_i)}"
    ))
fig.update_layout(title="Loss function for SGD with different (dynamic) step and number of max iterations", xaxis_title="Iteration", yaxis_title="Loss (MSE)")
fig.show()

In [ ]:
loss_list = []
for eta_i, max_iter_i in product(eta_list, max_iter_list):
    temp_class = LinearReg(optimizer_type="full", gd_type="stochastic", batch_size=2e5, max_iter=max_iter_i, eta=eta_i) # batch sgd
    temp_class.fit(X=x_train, y=y_train)
    loss_list.append(temp_class.loss_history)
    pprint(f"number of iterations: {temp_class.final_iterations}")
    pprint(f"loss function on test: {temp_class.calc_loss(X=x_test, y=y_test)}")
    pprint(f"r2 on test: {r2_score(y_test, temp_class.predict(x_test))}")

In [ ]:
fig = go.Figure()
for i, (eta_i, max_iter_i) in enumerate(product(eta_list, max_iter_list)):
    fig.add_trace(go.Scatter(
        y=loss_list[i], mode="lines", name=f"eta={eta_i}, max_iter={int(max_iter_i)}"
    ))
fig.update_layout(title="Loss function for Batch-SGD (2e5) with different (static) step and number of max iterations", xaxis_title="Iteration", yaxis_title="Loss (MSE)")
fig.show()

In [ ]:
loss_list = []
for eta_i, max_iter_i in product(eta_list, max_iter_list):
    temp_class = LinearReg(optimizer_type="full", gd_type="stochastic", batch_size=2e5, max_iter=max_iter_i, eta=eta_i, eta_type="dynamic") # batch sgd
    temp_class.fit(X=x_train, y=y_train)
    loss_list.append(temp_class.loss_history)
    pprint(f"number of iterations: {temp_class.final_iterations}")
    pprint(f"loss function on test: {temp_class.calc_loss(X=x_test, y=y_test)}")
    pprint(f"r2 on test: {r2_score(y_test, temp_class.predict(x_test))}")

In [ ]:
fig = go.Figure()
for i, (eta_i, max_iter_i) in enumerate(product(eta_list, max_iter_list)):
    fig.add_trace(go.Scatter(
        y=loss_list[i], mode="lines", name=f"eta={eta_i}, max_iter={int(max_iter_i)}"
    ))
fig.update_layout(title="Loss function for Batch-SGD (2e5) with different (dynamic) step and number of max iterations", xaxis_title="Iteration", yaxis_title="Loss (MSE)")
fig.show()

In [286]:
loss_list = []
for eta_i, max_iter_i in product(eta_list, max_iter_list):
    temp_class = LinearReg(optimizer_type="momentum", gd_type="stochastic", batch_size=1e5, alpha_momentum=0.8, max_iter=max_iter_i, eta=eta_i) # momentum method
    temp_class.fit(X=x_train, y=y_train)
    loss_list.append(temp_class.loss_history)
    pprint(f"number of iterations: {temp_class.final_iterations}")
    pprint(f"loss function on test: {temp_class.calc_loss(X=x_test, y=y_test)}")
    pprint(f"r2 on test: {r2_score(y_test, temp_class.predict(x_test))}")

 26%|██▌       | 522/2000 [00:15<00:43, 34.14it/s]


>> number of iterations: 522
>> loss function on test: 0.27207436811933206
>> r2 on test: 0.5678946567344157


100%|██████████| 1000/1000 [00:27<00:00, 35.82it/s]


>> number of iterations: 999
>> loss function on test: 0.2720746578532032
>> r2 on test: 0.5678941965824555


100%|██████████| 2000/2000 [00:57<00:00, 34.70it/s]


>> number of iterations: 1999
>> loss function on test: 0.2720728367614641
>> r2 on test: 0.5678970888191497


In [287]:
fig = go.Figure()
for i, (eta_i, max_iter_i) in enumerate(product(eta_list, max_iter_list)):
    fig.add_trace(go.Scatter(
        y=loss_list[i], mode="lines", name=f"eta={eta_i}, max_iter={int(max_iter_i)}"
    ))
fig.update_layout(title="Loss function for Momentum method with different (static) step and number of max iterations", xaxis_title="Iteration", yaxis_title="Loss (MSE)")
fig.show()

In [289]:
loss_list = []
for eta_i, max_iter_i in product([1., 5e-1], max_iter_list):
    temp_class = LinearReg(optimizer_type="adagrad", gd_type="stochastic", batch_size=2e5, eps_adagrad=1e-8, max_iter=max_iter_i, eta_type="dynamic", eta=eta_i) # adagard method
    temp_class.fit(X=x_train, y=y_train)
    loss_list.append(temp_class.loss_history)
    pprint(f"number of iterations: {temp_class.final_iterations}")
    pprint(f"loss function on test: {temp_class.calc_loss(X=x_test, y=y_test)}")
    pprint(f"r2 on test: {r2_score(y_test, temp_class.predict(x_test))}")

100%|██████████| 1000/1000 [00:38<00:00, 25.84it/s]


>> number of iterations: 999
>> loss function on test: 0.272072695123777
>> r2 on test: 0.5678973137664749


100%|██████████| 2000/2000 [01:16<00:00, 26.02it/s]


>> number of iterations: 1999
>> loss function on test: 0.27207705227651663
>> r2 on test: 0.567890393787231


100%|██████████| 1000/1000 [00:38<00:00, 26.15it/s]


>> number of iterations: 999
>> loss function on test: 0.2720742478193561
>> r2 on test: 0.5678948477934367


100%|██████████| 2000/2000 [01:16<00:00, 26.04it/s]

>> number of iterations: 1999
>> loss function on test: 0.27207810791868764
>> r2 on test: 0.5678887172286275


In [290]:
fig = go.Figure()
for i, (eta_i, max_iter_i) in enumerate(product([1., 5e-1], max_iter_list)):
    fig.add_trace(go.Scatter(
        y=loss_list[i], mode="lines", name=f"eta={eta_i}, max_iter={int(max_iter_i)}"
    ))
fig.update_layout(title="Loss function for Adagrad method with different (static) step and number of max iterations", xaxis_title="Iteration", yaxis_title="Loss (MSE)")
fig.show()

In [291]:
loss_list = []
for eta_i, max_iter_i in product([1., 5e-1], max_iter_list):
    temp_class = LinearReg(optimizer_type="rmsprop", gd_type="stochastic", batch_size=1e5, eps_adagrad=1e-8, alpha_rmsprop=0.99, max_iter=max_iter_i, eta=eta_i) # rmsprop method
    temp_class.fit(X=x_train, y=y_train)
    loss_list.append(temp_class.loss_history)
    pprint(f"number of iterations: {temp_class.final_iterations}")
    pprint(f"loss function on test: {temp_class.calc_loss(X=x_test, y=y_test)}")
    pprint(f"r2 on test: {r2_score(y_test, temp_class.predict(x_test))}")

100%|██████████| 1000/1000 [00:28<00:00, 34.87it/s]


>> number of iterations: 999
>> loss function on test: 2.3607169792983242
>> r2 on test: -2.7492632170524356


100%|██████████| 2000/2000 [00:57<00:00, 34.98it/s]


>> number of iterations: 1999
>> loss function on test: 2.185363682273254
>> r2 on test: -2.4707691526261364


100%|██████████| 1000/1000 [00:28<00:00, 34.96it/s]


>> number of iterations: 999
>> loss function on test: 0.6630334166679439
>> r2 on test: -0.05302195163123846


100%|██████████| 2000/2000 [00:57<00:00, 35.06it/s]

>> number of iterations: 1999
>> loss function on test: 0.7489942705109406
>> r2 on test: -0.18954397872987117


In [292]:
fig = go.Figure()
for i, (eta_i, max_iter_i) in enumerate(product([1., 5e-1], max_iter_list)):
    fig.add_trace(go.Scatter(
        y=loss_list[i], mode="lines", name=f"eta={eta_i}, max_iter={int(max_iter_i)}"
    ))
fig.update_layout(title="Loss function for RMSProp method with different (static) step and number of max iterations", xaxis_title="Iteration", yaxis_title="Loss (MSE)")

**Задание 5 (6 балла)**. Постройте графики (на одной и той же картинке) зависимости величины функции потерь от номера итерации для всех реализованных видов стохастического градиентного спусков. Сделайте выводы о скорости сходимости различных модификаций градиентного спуска.

Не забывайте о том, что должны получиться *красивые* графики!

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

**Задание 6 (бонус) (0.01 балла)**.  Вставьте картинку с вашим любимым мемом в этот Jupyter Notebook